In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib as plt
import glob
from tqdm import tqdm
import geopandas as gpd
import folium
import random
import matplotlib.pyplot as plt
from folium import plugins
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Obtener la ruta base (carpeta) de este archivo
ruta_base = os.path.dirname(os.path.abspath('mapas'))

# Definir el nombre de la carpeta donde se encuentran los archivos CSV
nombre_carpeta = 'Basesdatos'

# Construir las rutas completas de los archivos CSV dentro de la carpeta
ruta_df_completo = os.path.join(ruta_base, nombre_carpeta, 'df_final.xlsx')
ruta_df_final = os.path.join(ruta_base, nombre_carpeta, 'final.csv')

# Cargar archivo
df_completo = pd.read_excel(ruta_df_completo)

# Eliminar columnas no necesarias
df_completo = df_completo.drop(columns = ['Unnamed: 0','geometry'])

# Inprimir DataFrame
df_completo.head()

Longitud  Latitud  Estación   Año  Mes  Dia  Hora  NO (µg/m³)  NO2 (µg/m³)  \
0    2.2045  41.4039         4  2019    4    2     1           1           22   
1    2.2045  41.4039         4  2019    4    2     2           1           37   
2    2.2045  41.4039         4  2019    4    2     3           3           75   
3    2.2045  41.4039         4  2019    4    2     4           1           57   
4    2.2045  41.4039         4  2019    4    2     5           1           45   

   PM10 (µg/m³)  NOx (µg/m³)  
0            19           22  
1            20           38  
2            27           79  
3            33           59  
4            30           46

In [3]:
# Comprobar número de estaciones
set(df_completo['Estación'])

{4, 42, 43, 44, 50, 54, 57, 58, 59, 60, 61}

In [4]:
# Cargar archivos para realizar el mapa
datos = pd.read_csv(ruta_df_final)

In [5]:
# Convertir la columna 'WKT' a objetos de geometría
datos['geometry'] = gpd.GeoSeries.from_wkt(datos['WKT'])

# Crear un GeoDataFrame a partir de df_completo
gdf_completo = gpd.GeoDataFrame(datos, geometry='geometry')

In [6]:
# Comprobar información general del DataFrame
gdf_completo.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   WKT             11 non-null     object  
 1   Area            11 non-null     float64 
 2   Perim           11 non-null     float64 
 3   Coord_X         11 non-null     float64 
 4   Coord_Y         11 non-null     float64 
 5   Estacio         11 non-null     int64   
 6   Longitud        11 non-null     float64 
 7   Latitud         11 non-null     float64 
 8   Codi_districte  11 non-null     int64   
 9   Nom_districte   11 non-null     object  
 10  Codi_barri      11 non-null     int64   
 11  Nom_barri       11 non-null     object  
 12  geometry        11 non-null     geometry
dtypes: float64(6), geometry(1), int64(3), object(3)
memory usage: 1.2+ KB


In [7]:
# Eliminar columnas inncesarias
gdf_completo = gdf_completo.drop(columns= ['WKT','Area','Perim','Coord_X','Coord_Y','Longitud','Latitud','Codi_districte',	'Nom_districte','Codi_barri','Nom_barri'], axis = 1)

# Renonmbrar la columna Estación
gdf_completo = gdf_completo.rename(columns={'Estacio':'Estación'})

# Modificar la columna Estación a int
gdf_completo["Estación"] = gdf_completo["Estación"].astype(int)

# Inprimir DataFrame
gdf_completo

Estación                                           geometry
0          4  MULTIPOLYGON (((432519.740 4584126.948, 432484...
1         42  MULTIPOLYGON (((427670.088 4582617.520, 427803...
2         43  MULTIPOLYGON (((430774.340 4581547.357, 430862...
3         44  MULTIPOLYGON (((430603.979 4583791.301, 430509...
4         50  MULTIPOLYGON (((431850.502 4580899.181, 431666...
5         54  MULTIPOLYGON (((429684.497 4586274.249, 429688...
6         57  MULTIPOLYGON (((427326.314 4580300.351, 427287...
7         58  MULTIPOLYGON (((427242.281 4585693.629, 427246...
8         59  MULTIPOLYGON (((431180.844 4583582.266, 431167...
9         60  MULTIPOLYGON (((430592.015 4586871.255, 430567...
10        61  MULTIPOLYGON (((431228.166 4580413.202, 431212...

In [8]:
# Realizar el merge basado en la columna 'Estación'
merged_df = pd.merge(df_completo, gdf_completo[['Estación', 'geometry']], on='Estación', how='left')

# Asignar la columna 'geometry' del GeoDataFrame al DataFrame original
df_completo['geometry'] = merged_df['geometry']

# Inprimir DataFrame
df_completo.head()

Longitud  Latitud  Estación   Año  Mes  Dia  Hora  NO (µg/m³)  NO2 (µg/m³)  \
0    2.2045  41.4039         4  2019    4    2     1           1           22   
1    2.2045  41.4039         4  2019    4    2     2           1           37   
2    2.2045  41.4039         4  2019    4    2     3           3           75   
3    2.2045  41.4039         4  2019    4    2     4           1           57   
4    2.2045  41.4039         4  2019    4    2     5           1           45   

   PM10 (µg/m³)  NOx (µg/m³)  \
0            19           22   
1            20           38   
2            27           79   
3            33           59   
4            30           46   

                                            geometry  
0  MULTIPOLYGON (((432519.740 4584126.948, 432484...  
1  MULTIPOLYGON (((432519.740 4584126.948, 432484...  
2  MULTIPOLYGON (((432519.740 4584126.948, 432484...  
3  MULTIPOLYGON (((432519.740 4584126.948, 432484...  
4  MULTIPOLYGON (((432519.740 4584126.948, 432484...

In [9]:
#df_completo.to_excel('df_limpia.xlsx')

In [10]:
# Suponiendo que 'gdf_completo' es el GeoDataFrame con los datos proporcionados
gdf_completo = gpd.GeoDataFrame(df_completo, crs="25831", geometry="geometry")

# Transformar las geometrías al CRS deseado (WGS 84 - EPSG:4326)
gdf_completo = gdf_completo.to_crs(epsg=4326)

In [11]:
# Filtrar para un día, mes, año y hora específicos
dia = 17
mes = 6
año = 2022
hora =21

df_filtrado = gdf_completo[
    (gdf_completo['Dia'] == dia) &
    (gdf_completo['Mes'] == mes) &
    (gdf_completo['Año'] == año) &
    (df_completo['Hora'] == hora)
]

In [12]:
# Obtener la cantidad de polígonos a graficar
total_poligonos = len(df_filtrado)
total_poligonos

11

In [13]:
# Crear el mapa de folium
m = folium.Map(location=[41.4279, 2.16992], zoom_start=12.4, tiles="CartoDB positron")

# Iterar a través de las filas del GeoDataFrame
for _, r in df_filtrado.iterrows():
    # Obtener el objeto GeoJson y agregarlo al mapa
    geo_j = folium.GeoJson(data=r["geometry"], style_function=lambda x: {"fillColor": "black"})
    geo_j.add_to(m)  # Agregar la capa al mapa principal 'm'

# Mostrar el mapa
m

In [14]:
# Crear el mapa de folium
m2 = folium.Map(location=[41.3879, 2.16992], zoom_start=12, tiles="CartoDB positron")

# Definir función para asignar colores en función del valor de "PM10"
def get_color(pm10_value):
    if pm10_value <= 15:
        return "mediumseagreen"  # Verde para valores bajos de PM10
    elif 15 < pm10_value <= 30:
        return "lightgreen"  # Amarillo para valores moderados de PM10
    elif 30 < pm10_value <= 60:
        return "lightyellow"  # Amarillo para valores moderados de PM10
    elif 60 < pm10_value <= 90:
        return "orangered"  # Naranja para valores altos de PM10
    else:
        return "red"  # Rojo para valores muy altos de PM10

# Iterar a través de las filas del GeoDataFrame
for _, r in df_filtrado.iterrows():
    # Obtener el valor de "PM10"
    pm10_value = r["PM10 (µg/m³)"]

    # Crear el objeto GeoJson y agregarlo al mapa con estilo de color basado en la función get_color
    folium.GeoJson(
        data=r["geometry"],
        style_function=lambda x, color=get_color(pm10_value): {
            "fillColor": color,
            "color": "black",
            "weight": 2,
            "fillOpacity": 0.4,
        },
        tooltip=f"PM10: {pm10_value}",
    ).add_to(m2)  # Agregar la capa al mapa principal 'm2'

# Añadir la escala de color simple
colormap = folium.LinearColormap(
    colors=["mediumseagreen", "lightgreen", "lightyellow", "orangered", "red"],
    vmin=0,
    vmax=100,
    caption="PM10 (µg/m³)"
)
colormap.add_to(m2)

# Agregar la leyenda de colores
colormap.caption = "PM10 (µg/m³)"
colormap.add_to(m2)

plugins.Geocoder().add_to(m2)
# Mostrar el mapa
m2